In [4]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# from dataprep.eda import plot, plot_correlation, plot_missing

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [5]:
import os
path = os.path.dirname(os.getcwd())
path = os.path.join(path, "ProcessedData")

In [6]:
train = pd.read_csv(os.path.join(path,"merged_train.csv"))
test = pd.read_csv(os.path.join(path,"merged_test.csv"))

In [81]:
class preprocess:
    
    def __init__(self,df):
        self.df = df
        self.non_unique = np.array(['자격유형', '공급유형', '임대건물구분','임대보증금', '임대료', '전용면적', '전용면적별세대수'])
        self.apt = df[df['임대건물구분']=='아파트']
        self.store= df[df['임대건물구분']=='상가']
        
    def new_y(self):
        try:
            self.df['등록차량수/총세대수'] = self.df.apply(lambda x : x['등록차량수']/x['총세대수'], axis=1)
            self.df['등록차량수/단지내주차면수'] = self.df.apply(lambda x : x['등록차량수']/x['단지내주차면수'], axis=1)
            self.y = np.array(['등록차량수','등록차량수/단지내주차면수','등록차량수/총세대수'])
        except:
            pass
        
    def no_duplicates(self): #첫번째 방안
        self.newDF = self.df.drop(self.non_unique,axis=1).drop_duplicates().reset_index(drop=True)

        try:
            self.newDF = self.newDF.drop(self.y,axis=1)
        except:
            pass
        
        return self.newDF
    
    def merge(self,col,standard,oldDF,newDF,kind):
        
        sample = oldDF.groupby('단지코드').describe()[col].reset_index()
        for i in sample.index:
            for j in standard:
                code = sample.loc[i,'단지코드']
                val = sample.loc[i,j]
                ind = newDF[newDF['단지코드']==code].index
                newDF.loc[ind,kind+col+j] = val
        return newDF
    
    def category(self,x,dic):
        for i in dic.keys():
            if x in dic[i]:
                return int(i)
        
    def grouping(self,col,dic,newDF,count=None):
        self.df[col+'카테고리'] = self.df[col].apply(lambda x : preprocess.category(self,x,dic))
        sample = pd.DataFrame(self.df.groupby('단지코드')[col+'카테고리'].unique()).reset_index()
        for i in dic.keys():
            newDF[col+str(i)]=0
        for i in sample.index:
            code = sample.loc[i,'단지코드']
            val = sample.loc[i,col+'카테고리']
            ind = newDF[self.newDF['단지코드']==code].index
            for j in val:
                if count == None:
                    newDF.loc[ind,col+str(j)] = 1                   
        return newDF
                    
    
    def process(self):
        
        #아파트 임대료, 임대보증금
#         self.newDF = preprocess.merge(self,'임대료',['min','25%','50%','75%','max'],self.apt,self.newDF,'apt')
#         self.newDF = preprocess.merge(self,'임대보증금',['min','25%','50%','75%','max'],self.apt,self.newDF,'apt')

#         #상가 임대료, 임대보증금
#         self.newDF = preprocess.merge(self,'임대료',['mean','std'],self.store,self.newDF,'store')
#         self.newDF = preprocess.merge(self,'임대보증금',['mean','std'],self.store,self.newDF,'store')

#         #상가 없는 경우 임대료 임대보증금 0으로 채움
#         self.newDF = self.newDF.replace(np.nan, 0)

#         # 자격유형 0,1 indicator variable
#         dic = {'1': 'A' ,'2':['E','K'],'3':['D'],'4':['L','M','N','O'],'5':['B','H','I','G'],'6':'J','7':['C','F']}
#         self.newDF = preprocess.grouping(self,'자격유형',dic,count=None)
        
        # 지역
        dic = {'1':['경상남도', '전라북도', '강원도', '광주광역시', '충청남도', '제주특별자치도', '울산광역시', '충청북도', '전라남도', '경상북도', '세종특별자치시'],'2':['경기도' '서울특별시' '대구광역시' '부산광역시' '대전광역시']}
        self.newDF = preprocess.grouping(self,'지역',dic,self.newDF,count=None)
        
        
        return self.newDF

In [83]:
dic.keys()

dict_keys(['1', '2'])

In [82]:
pr = preprocess(train)
pr.new_y()
pr.no_duplicates()
pr.process()

,단지코드,총세대수,지역,공가수,지하철역,버스정류장,단지내주차면수,단지명,도로명주소,연면적,위도,경도,subway_name,subway_dist,환승역 수,총인구수,세대당_인구,남/여비율,남/여_0~19세,남/여_20~39세,남/여_40~69세,남/여_70세이상,0~19세_비율,20~39세_비율,40~69세_비율,70세이상_비율,자격유형카테고리,지역카테고리,지역1,지역2,지역1.0,지역nan
0,C2515,545,경상남도,17.0000,0.0000,3.0000,624.0000,미수휴먼시아,경상남도 통영시 미우지해안로 107,"1,213.8700",128.3965,34.8242,다대포해수욕장역,0.6117,1,"11,129.0000",2.3000,0.9700,1.7612,1.0366,0.9840,0.5888,0.1956,0.1850,0.5221,0.0972,1,1.0000,0,0,1.0000,nan
1,C1407,1216,대전광역시,13.0000,1.0000,1.0000,"1,285.0000",도안 휴먼시아 4단지,대전광역시 유성구 상대로 40,"4,636.4100",127.3374,36.3466,유성온천역,0.0082,1,"45,840.0000",2.4200,0.9200,2.1671,0.9294,0.8716,0.6995,0.2288,0.2761,0.4425,0.0527,1,nan,0,0,nan,1.0000
2,C1945,755,경기도,6.0000,1.0000,3.0000,734.0000,호평마을 주공휴먼시아,경기도 남양주시 늘을3로 8,"6,017.7600",127.2408,37.6584,평내호평역,0.0063,1,"57,499.0000",2.6500,0.9500,1.9906,0.9644,0.9377,0.6974,0.2349,0.2390,0.4468,0.0793,5,nan,0,0,nan,1.0000
3,C1470,696,전라북도,14.0000,0.0000,2.0000,645.0000,장신휴먼시아1단지,전라북도 익산시 오산면 선화로4길 66,"6,611.9000",126.9312,35.9467,연산역,0.3785,2,"7,783.0000",2.0000,0.9800,1.8224,1.1859,1.0494,0.6716,0.1429,0.1798,0.4903,0.1871,1,1.0000,0,0,1.0000,nan
4,C1898,566,전라북도,9.0000,0.0000,6.0000,517.0000,장신휴먼시아2단지,전라북도 익산시 오산면 선화로4길 24-9,"6,749.2000",126.9320,35.9486,연산역,0.3766,2,"7,783.0000",2.0000,0.9800,1.8224,1.1859,1.0494,0.6716,0.1429,0.1798,0.4903,0.1871,1,1.0000,0,0,1.0000,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,C1698,460,경상남도,1.0000,0.0000,3.0000,155.0000,진해자은주공아파트,경상남도 창원시 진해구 진해대로901번길 11,528.0200,128.7081,35.1500,봉황역,0.1833,1,"13,389.6667",2.2367,1.0300,2.1214,1.1855,0.9905,0.6585,0.1922,0.2466,0.4686,0.0926,3,1.0000,0,0,1.0000,nan
484,C1004,521,충청남도,3.0000,0.0000,2.0000,153.0000,주공6단지2차아파트,충청남도 천안시 서북구 쌍용대로 289-28,948.7600,127.1337,36.8240,두정역,0.0181,1,"21,296.0000",1.6550,1.1400,1.7826,1.2580,1.1829,0.6887,0.0968,0.3980,0.4273,0.0778,3,1.0000,0,0,1.0000,nan
485,C1875,1003,충청남도,5.0000,0.0000,8.0000,192.0000,쌍용주공7단지1차아파트,충청남도 천안시 서북구 쌍용11길 57,"1,009.0500",127.1261,36.8026,봉명역,0.0100,1,"23,441.6667",2.3833,0.9500,1.7883,1.0584,0.9108,0.6012,0.1979,0.2855,0.4517,0.0649,3,1.0000,0,0,1.0000,nan
486,C2212,697,강원도,0.0000,0.0000,4.0000,128.0000,석사주공3단지아파트,강원도 춘천시 후석로228번길 47,12.0000,127.7577,37.8711,남춘천역,0.0346,1,"15,898.0000",2.1800,0.9367,1.9287,1.0658,0.9135,0.6272,0.1622,0.2705,0.4435,0.1238,3,1.0000,0,0,1.0000,nan


In [13]:
pr1 = preprocess(test)
pr1.new_y()
pr1.no_duplicates()
pr1.process()

,단지코드,총세대수,지역,공가수,지하철역,버스정류장,단지내주차면수,단지명,도로명주소,연면적,위도,경도,subway_name,subway_dist,환승역 수,총인구수,세대당_인구,남/여비율,남/여_0~19세,남/여_20~39세,남/여_40~69세,남/여_70세이상,0~19세_비율,20~39세_비율,40~69세_비율,70세이상_비율
0,C1072,754,경기도,14,0.0000,2,683,광명역세권 휴먼시아2단지,경기도 광명시 서원로 25,13.3200,126.8834,37.4294,광명역,0.0130,1,"41,315.0000",2.5800,0.9800,1.9091,1.0299,0.9859,0.7089,0.2031,0.2745,0.4513,0.0710
1,C1128,1354,경기도,9,0.0000,3,1216,광명역세권 휴먼시아1단지,경기도 광명시 성채안로 26,"3,773.7700",126.8873,37.4346,석수역,0.0150,1,"41,315.0000",2.5800,0.9800,1.9091,1.0299,0.9859,0.7089,0.2031,0.2745,0.4513,0.0710
2,C1456,619,부산광역시,18,0.0000,16,547,안락휴먼시아,부산광역시 동래구 충렬대로447번길 38,"6,035.7800",129.1083,35.1982,안락역,0.0079,1,"21,096.5000",2.3050,0.9600,1.8610,1.1094,0.9045,0.7829,0.1506,0.2248,0.4973,0.1273
3,C1840,593,전라북도,7,0.0000,3,543,장신휴먼시아3단지,전라북도 익산시 오산면 선화로4길 14,"6,006.8700",126.9330,35.9501,연산역,0.3748,2,"7,783.0000",2.0000,0.9800,1.8224,1.1859,1.0494,0.6716,0.1429,0.1798,0.4903,0.1871
4,C1332,1297,경기도,11,0.0000,2,1112,봇들마을 6단지,경기도 성남시 분당구 동판교로 212,361.7890,127.1182,37.4009,판교역,0.0093,2,"19,109.5000",2.8600,0.9550,1.8566,0.9241,0.9454,0.7738,0.2206,0.2526,0.4552,0.0716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,C2456,349,제주특별자치도,17,0.0000,4,270,서귀포성산LH,제주특별자치도 서귀포시 성산읍 고성동서로 33,40.9200,126.9102,33.4501,녹동역,1.6568,1,"15,478.0000",1.9000,1.1000,2.1004,1.1739,1.2671,0.6607,0.1256,0.1841,0.5176,0.1726
143,C1266,596,충청북도,35,0.0000,1,593,충주호암A4-1블록(국민임대),충청북도 충주시 호암수청2로 56,207.4500,127.9271,36.9552,여주역,0.4433,1,"18,468.0000",2.5100,0.9800,1.8931,1.0803,0.9702,0.6374,0.2386,0.2595,0.4373,0.0645
144,C2152,120,강원도,9,0.0000,1,40,화천신읍(공공실버) 영구임대,강원도 화천군 화천읍 산천어길 196,0.0000,127.7030,38.1144,춘천역,0.2303,1,"8,365.0000",2.1000,1.0900,1.9810,1.4255,1.1577,0.6088,0.1707,0.2589,0.4253,0.1450
145,C1267,675,경상남도,38,0.0000,1,467,창원가포1단지,경상남도 창원시 마산합포구 가포본동5길 37,"2,718.8181",128.5837,35.1584,수로왕릉역,0.2979,1,960.0000,1.6200,1.1100,2.6250,1.2233,1.1741,0.8427,0.0833,0.2385,0.5073,0.1708
